In [164]:
import pandas as pd
import json
import os

# preview the data
path = '/Users/wzd0011/Dropbox/miscellanea/Job hunting/follow-ups/ID Analytics/data/ida_wrangling_exercise_data.2017-02-13.jsonl'
data = []
with open(path,'r') as f:
    for line in f:
        data.append(json.loads(line))


# Problem 1
Start by making a list of all of the nested named fields that appear in any record. 
Concatenate nested field names using a period '.' to defind named fields for nested records. 
Present the list in alphabetical order.

In [2]:
import json
print(json.__file__) #this would be important to know if it works

C:\Users\wzd0011\AppData\Local\Continuum\Anaconda3\lib\json\__init__.py


In [5]:
'''import pip 
def install(package):
    pip.main(['install', package])

# Example
if __name__ == '__main__':
    install('jsonlines')
'''
    
import jsonlines

def flat_keys(obj, new_obj1={}, keys=[]):
    for key, value in obj.items():
        if isinstance(value, dict):
            # call the function again if the value is a dict
            # we go one step deeper: obj[key]
            # give the data_name (by reference, so each call edit the *same* object)
            # give to used keys: keys + [key]
            flat_keys(obj[key], new_obj1, keys + [key])
        else:
            new_obj1['.'.join(keys + [key])] = value
    return new_obj1

data_name_list = []

with jsonlines.open(path) as f:
    for item in f:
        new_obj = flat_keys(item,{},[])
        if new_obj not in data_name_list:
            data_name_list.append(new_obj)
    
all_keys = set().union(*(d.keys() for d in data_name_list))
print(sorted(all_keys))

['address', 'address.city', 'address.state', 'address.street', 'address.zip', 'dob', 'email', 'id', 'name', 'name.firstname', 'name.lastname', 'name.middlename', 'phone', 'record_date', 'ssn']


# Problem 2
Answer the following questions for each field in your list from question 1.
- What percentage of the records contain the field?
- What are the five most common values of the field?

In [6]:
import operator
data_name_list_total = {}

with jsonlines.open(path) as f:
    for item in f:
        new_obj = flat_keys(item,{},[])
        new_obj = sorted(new_obj.items(), key=operator.itemgetter(0))
        for name_obj in new_obj:
            if name_obj[0] in data_name_list_total:
                data_name_list_total[name_obj[0]] += 1
            else:
                data_name_list_total[name_obj[0]] = 1

for name_obj, occurences in sorted(data_name_list_total.items()):
    print(name_obj, ':', occurences)            

address : 75193
address.city : 61233
address.state : 61233
address.street : 61233
address.zip : 61233
dob : 143874
email : 130880
id : 150000
name : 43058
name.firstname : 105004
name.lastname : 105004
name.middlename : 43669
phone : 140261
record_date : 150000
ssn : 142444


In [66]:
print("Percentage of the records contain the field:")
for name_obj, occurences in sorted(data_name_list_total.items()):
    print(name_obj, ':', "{0:.1f}%".format( occurences/len(data) * 100)) 

Percentage of the records contain the field:
address : 50.1%
address.city : 40.8%
address.state : 40.8%
address.street : 40.8%
address.zip : 40.8%
dob : 95.9%
email : 87.3%
id : 100.0%
name : 28.7%
name.firstname : 70.0%
name.lastname : 70.0%
name.middlename : 29.1%
phone : 93.5%
record_date : 100.0%
ssn : 95.0%


In [65]:
sort_com = sorted(data_name_list_total.items(), key=lambda x: x[1], reverse=True)
print('The five most common values of the field: ')
for item in sort_com[:5]:
    print(item[0])

The five most common values of the field: 
record_date
id
dob
ssn
phone


# Problem 3
How many distinct first names appear in this data set? Explain your procedure for identifying distinct first names.

In [91]:
# similar approach in problem 2, but only count the filed equals to "name.firstname"
import operator
data_name_list_total = []
count = 0
with jsonlines.open(path) as f:
    for item in f:
        new_obj = flat_keys(item,{},[])
        new_obj = sorted(new_obj.items(), key=operator.itemgetter(0))
        for name_obj in new_obj:
            if name_obj[0] == "name.firstname": 
                if name_obj[1] not in data_name_list_total:
                    data_name_list_total.append(name_obj[1])
                    count += 1

print("Number of distinct first names: ", count)

Number of distinct first names:  690


# Problem 4
How many distinct street names appear in this data set? Explain your procedure for identifying distinct street names.

In [92]:
# similar approach in problem 2, but only count the filed equals to "address.street"
import operator
data_name_list_total = []
count = 0
with jsonlines.open(path) as f:
    for item in f:
        new_obj = flat_keys(item,{},[])
        new_obj = sorted(new_obj.items(), key=operator.itemgetter(0))
        for name_obj in new_obj:
            if name_obj[0] == "address.street": 
                if name_obj[1] not in data_name_list_total:
                    data_name_list_total.append(name_obj[1])
                    count += 1

print("Number of distinct street names: ", count)

Number of distinct street names:  61233


# Problem 5
What are the 5 most common US area codes in the phone number field? Explain your approach to identify the US area codes in this data set.

In [94]:
# First to check the phone belong to US and then extract the three digits before the dash line or the dot as the area codes
import operator
phone_list = []
with jsonlines.open(path) as f:
    for item in f:
        new_obj = flat_keys(item,{},[])
        new_obj = sorted(new_obj.items(), key=operator.itemgetter(0))
        for name_obj in new_obj:
            if name_obj[0] == "phone": 
                if name_obj[1] not in phone_list:
                    phone_list.append(name_obj[1])
print(len(phone_list))                    

140260


In [115]:
# delete record of phone number beginning with +xx(x) where xx is not 1 
phone_list_filter1 = list(filter(lambda x: not x.startswith("+") and not x[1]==1, phone_list))

In [135]:
# the second filter make sure the string start with the area codes 
phone_list_filter2 = []
for item in phone_list_filter1:
    if item.startswith("1-"):
        phone_list_filter2.append(item[2:]) 
    elif item.startswith("("):
        phone_list_filter2.append(item[1:]) 
    elif item.startswith("0") and len(item)==10:    
        phone_list_filter2.append(item[1:]) 

In [163]:
# read the first three digits
phone_list_filter3 = []
for item in phone_list_filter2:
    first_three = item[:3]
    phone_list_filter3.append(first_three)

# print the 5 most common US area code 
from collections import Counter
code_data = Counter(phone_list_filter3)
print('The five most common area codes: ')
for item in code_data.most_common(5):
    print(item[0])

The five most common area codes: 
226
012
380
469
156
